In [1]:
]activate .

In [2]:
using DifferentialEquations
using StaticArrays
using BenchmarkTools

@inbounds @inline function ż(z, p, t)
    A, B, D = p
    p₀, p₂ = z[SVector{2}(1:2)]
    q₀, q₂ = z[SVector{2}(3:4)]

    return SVector{4}(
        -A * q₀ - 3 * B / √2 * (q₂^2 - q₀^2) - D * q₀ * (q₀^2 + q₂^2),
        -q₂ * (A + 3 * √2 * B * q₀ + D * (q₀^2 + q₂^2)),
        A * p₀,
        A * p₂
    )
end

@inbounds @inline function ṗ(p, q, params, t)
    A, B, D = params
    dp1 = -A * q[1] - 3 * B / √2 * (q[2]^2 - q[1]^2) - D * q[1] * (q[1]^2 + q[2]^2)
    dp2 = -q[2] * (A + 3 * √2 * B * q[1] + D * (q[1]^2 + q[2]^2))
    return SVector{2}(dp1, dp2)
end

@inbounds @inline function q̇(p, q, params, t)
    params.A * p
end

q̇ (generic function with 1 method)

In [3]:
q0 = SVector{2}([0.0, -4.363920590485035])
p0 = SVector{2}([10.923918825236079, -5.393598858645495])
z0 = vcat(p0, q0)
p = (A=1,B=0.55,D=0.4)

tspan = (0., 10.)

(0.0, 10.0)

In [4]:
prob1 = ODEProblem(ż, z0, tspan, p)
prob2 = DynamicalODEProblem(ṗ, q̇, p0, q0, tspan, p)

ODEProblem with uType Tuple{SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2}} and tType Float64. In-place: false
timespan: (0.0, 10.0)
u0: ([10.9239, -5.3936], [0.0, -4.36392])

In [5]:
@btime solve($prob1, Vern9(), abstol=1e-14, reltol=1e-14);
@btime solve($prob2, DPRKN12(), abstol=1e-14, reltol=1e-14);
@btime solve($prob2, KahanLi8(), dt=1e-2, maxiters=1e10);

  637.078 μs (39946 allocations: 1.55 MiB)
  292.822 μs (17753 allocations: 534.38 KiB)
  1.274 ms (110196 allocations: 3.05 MiB)


In [6]:
@btime solve($prob1, Vern9(), abstol=1e-14, reltol=1e-14, save_everystep=false);
@btime solve($prob2, DPRKN12(), abstol=1e-14, reltol=1e-14, save_everystep=false);
@btime solve($prob2, KahanLi8(), dt=1e-2, maxiters=1e10, save_everystep=false);

  596.457 μs (38974 allocations: 1.21 MiB)
  279.379 μs (17439 allocations: 463.05 KiB)
  1.203 ms (108191 allocations: 2.76 MiB)


In [7]:
function step_integ(integ, t)
    while integ.t < t
        step!(integ)
    end
end

function integ_benchmark(prob; args...)
    integ = init(prob; args..., save_everystep=false)
    while integ.t < prob.tspan[2]
        step!(integ)
    end
end

integ_benchmark (generic function with 1 method)

In [8]:
@btime init($prob1, Vern9(), abstol=1e-14, reltol=1e-14, save_everystep=false)
@btime step_integ(integ1, $tspan[2]) setup=(integ1=init($prob1, Vern9(), abstol=1e-14, reltol=1e-14, save_everystep=false))
@btime init($prob2, DPRKN12(), abstol=1e-14, reltol=1e-14, save_everystep=false)
@btime step_integ(integ2, $tspan[2]) setup=(integ2=init($prob2, DPRKN12(), abstol=1e-14, reltol=1e-14, save_everystep=false))
@btime init($prob2, KahanLi8(), dt=1e-2, maxiters=1e10, save_everystep=false)
@btime step_integ(integ3, $tspan[2]) setup=(integ3=init($prob2, KahanLi8(), dt=1e-2, maxiters=1e10, save_everystep=false))

  4.335 μs (88 allocations: 18.45 KiB)
  3.354 μs (223 allocations: 6.98 KiB)
  4.258 μs (95 allocations: 11.13 KiB)
  1.114 μs (69 allocations: 1.80 KiB)
  2.825 μs (79 allocations: 6.03 KiB)
  120.168 μs (10810 allocations: 281.53 KiB)


In [9]:
@btime integ_benchmark($prob1, alg=Vern9(), abstol=1e-14, reltol=1e-14)
@btime integ_benchmark($prob2, alg=DPRKN12(), abstol=1e-14, reltol=1e-14)
@btime integ_benchmark($prob2, alg=KahanLi8(), dt=1e-2, maxiters=1e10)

  616.037 μs (39457 allocations: 1.21 MiB)
  289.900 μs (17596 allocations: 465.30 KiB)
  1.251 ms (109192 allocations: 2.77 MiB)


In [10]:
tspan = (0., 100.)
prob1 = ODEProblem(ż, z0, tspan, p)
prob2 = DynamicalODEProblem(ṗ, q̇, p0, q0, tspan, p)

ODEProblem with uType Tuple{SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2}} and tType Float64. In-place: false
timespan: (0.0, 100.0)
u0: ([10.9239, -5.3936], [0.0, -4.36392])

In [11]:
@btime solve($prob1, Vern9(), abstol=1e-14, reltol=1e-14, save_everystep=false);
@btime solve($prob2, DPRKN12(), abstol=1e-14, reltol=1e-14, save_everystep=false);
@btime solve($prob2, KahanLi8(), dt=1e-2, maxiters=1e10, save_everystep=false);

  5.868 ms (386012 allocations: 11.80 MiB)
  2.779 ms (173906 allocations: 4.43 MiB)
  14.675 ms (1080082 allocations: 27.47 MiB)


In [12]:
@btime init($prob1, Vern9(), abstol=1e-14, reltol=1e-14, save_everystep=false)
@btime step_integ(integ1, $tspan[2]) setup=(integ1=init($prob1, Vern9(), abstol=1e-14, reltol=1e-14, save_everystep=false))
@btime init($prob2, DPRKN12(), abstol=1e-14, reltol=1e-14, save_everystep=false)
@btime step_integ(integ2, $tspan[2]) setup=(integ2=init($prob2, DPRKN12(), abstol=1e-14, reltol=1e-14, save_everystep=false))
@btime init($prob2, KahanLi8(), dt=1e-2, maxiters=1e10, save_everystep=false)
@btime step_integ(integ3, $tspan[2]) setup=(integ3=init($prob2, KahanLi8(), dt=1e-2, maxiters=1e10, save_everystep=false))

  4.384 μs (86 allocations: 18.34 KiB)
  5.858 ms (385920 allocations: 11.78 MiB)
  4.300 μs (94 allocations: 11.03 KiB)
  308.408 μs (19312 allocations: 502.92 KiB)
  2.825 μs (78 allocations: 5.94 KiB)
  14.674 ms (1080000 allocations: 27.47 MiB)


In [13]:
@btime integ_benchmark($prob1, alg=Vern9(), abstol=1e-14, reltol=1e-14)
@btime integ_benchmark($prob2, alg=DPRKN12(), abstol=1e-14, reltol=1e-14)
@btime integ_benchmark($prob2, alg=KahanLi8(), dt=1e-2, maxiters=1e10)

  6.154 ms (390834 allocations: 11.87 MiB)
  2.877 ms (175454 allocations: 4.45 MiB)
  15.110 ms (1090082 allocations: 27.62 MiB)
